<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego 
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej" 
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

In [ ]:
import random
import numpy as np
from torchvision import datasets, transforms

In [ ]:
!wget -O mnist.npz https://s3.amazonaws.com/img-datasets/mnist.npz

In [ ]:
# Let's read the mnist dataset

def load_mnist(path='mnist.npz'):
    with np.load(path) as f:
        x_train, _y_train = f['x_train'], f['y_train']
        x_test, _y_test = f['x_test'], f['y_test']
        
    x_train = x_train.reshape(-1, 28 * 28) / 255.
    x_test = x_test.reshape(-1, 28 * 28) / 255.
    
    y_train = np.zeros((_y_train.shape[0], 10))
    y_train[np.arange(_y_train.shape[0]), _y_train] = 1
    
    y_test = np.zeros((_y_test.shape[0], 10))
    y_test[np.arange(_y_test.shape[0]), _y_test] = 1

    return (x_train, y_train), (x_test, y_test)

(x_train, y_train), (x_test, y_test) = load_mnist()

## Exercise 1

In this exercise your task is to fill in the gaps in this code by implementing the backpropagation algorithm
Once this is done, you can run the network on the MNIST example and see how it performs. Feel free to play with the parameters. Your model should achieve 90%+ accuracy after a few epochs.


## Exercise 2 (Optional)

Implement a "fully vectorized" version, i.e. one using matrix operations instead of going over examples one by one within a minibatch.


In [ ]:
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    # Derivative of the sigmoid
    return sigmoid(z)*(1-sigmoid(z))

class Network(object):
    def __init__(self, sizes):
        # initialize biases and weights with random normal distr.
        # weights are indexed by target node first
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) 
                        for x, y in zip(sizes[:-1], sizes[1:])]
    def feedforward(self, a):
        # Run the network on a single case
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a
    
    def update_mini_batch(self, x_mini_batch, y_mini_batch, eta):
        # Update networks weights and biases by applying a single step
        # of gradient descent using backpropagation to compute the gradient.
        # The gradient is computed for a mini_batch.
        # eta is the learning rate
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in zip(x_mini_batch, y_mini_batch):
            delta_nabla_b, delta_nabla_w = self.backprop(x.reshape(784,1), y.reshape(10,1))
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(x_mini_batch))*nw 
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(x_mini_batch))*nb 
                       for b, nb in zip(self.biases, nabla_b)]
        
    def backprop(self, x, y):
        # For a single input (x,y) return a pair of lists.
        # First contains gradients over biases, second over weights.
        g = x
        gs = [g] # list to store all the gs, layer by layer
        fs = [] # list to store all the fs, layer by layer
        for b, w in zip(self.biases, self.weights):
            f = np.dot(w, g)+b
            fs.append(f)
            g = sigmoid(f)
            gs.append(g)
        # backward pass <- both steps at once
        dLdg = self.cost_derivative(gs[-1], y)
        dLdfs = []
        for w,g in reversed(list(zip(self.weights,gs[1:]))):
            dLdf = np.multiply(dLdg,np.multiply(g,1-g))
            dLdfs.append(dLdf)
            dLdg = np.matmul(w.T, dLdf)
        
        dLdWs = [np.matmul(dLdf,g.T) for dLdf,g in zip(reversed(dLdfs),gs[:-1])] # automatic here
        dLdBs = [np.sum(dLdf,axis=1).reshape(dLdf.shape[0],1) for dLdf in reversed(dLdfs)] # CHANGE: Need to sum here
    
        return (dLdBs,dLdWs)

    def evaluate(self, x_test_data, y_test_data):
        # Count the number of correct answers for test_data
        test_results = [(np.argmax(self.feedforward(x_test_data[i].reshape(784,1))), np.argmax(y_test_data[i]))
                        for i in range(len(x_test_data))]
        # return accuracy
        return np.mean([int(x == y) for (x, y) in test_results])
    
    def cost_derivative(self, output_activations, y):
        return (output_activations-y) 
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        x_train, y_train = training_data
        if test_data:
            x_test, y_test = test_data
        for j in range(epochs):
            for i in range(x_train.shape[0] // mini_batch_size):
                x_mini_batch = x_train[i*mini_batch_size:(i*mini_batch_size + mini_batch_size)] 
                y_mini_batch = y_train[i*mini_batch_size:(i*mini_batch_size + mini_batch_size)] 
                self.update_mini_batch(x_mini_batch, y_mini_batch, eta)
            if test_data:
                print("Epoch: {0}, Accuracy: {1}".format(j, self.evaluate(x_test, y_test)))
            else:
                print("Epoch: {0}".format(j))


network = Network([784,30,10])
network.SGD((x_train, y_train), epochs=10, mini_batch_size=100, eta=3., test_data=(x_test, y_test))



# Excercise 3 (optional)

Standart backpropagation method requires memorization of all outputs of all layers, which can take much of precious GPU memory.
Instead of doing that, one can memorize only a select few layers and then recompute the rest as they are needed.
Your task is to complete the code below to implement backpropagation with checkpoints. 
To keep things simple, use 1-example mini-batches (or, if you are bored, vectorize the code below)

In [ ]:
class NetworkWithCheckpoints(object):
    def __init__(self, sizes, checkpoints):
        # initialize biases and weights with random normal distr.
        # weights are indexed by target node first
        self.num_layers = len(sizes) - 1
        self.sizes = sizes 
        self.checkpoints = checkpoints
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) 
                        for x, y in zip(sizes[:-1], sizes[1:])]
        
    def feedforward(self, a):
        # Run the network on a single case
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a
    
    def feedforward_with_checkpoints(self, x):
        # Runs network on a single case, memorizing the activations of layers included in checkpoints.
        # Notice that gs (outputs of non-linearities) are shifted by one
        fs = []
        gs = [x]
        g = x
        for i, (w, b) in enumerate(zip(self.weights, self.biases)):
            f = np.dot(w, g) + b
            g = sigmoid(f)
            if i in self.checkpoints:
                fs.append(f)
                gs.append(g)
            else:
                fs.append(None)
                gs.append(None)
        return fs, gs, g

    
    def feedforward_between_layers(self, first_layer: int, last_layer: int, acc_f, acc_g):
        # feedforward input acc_g[first_layer] for layers [first_layer, last_layer)
        # memorizing their outputs in respective indexes of acc_f, acc_g
        
        for i in range(first_layer, last_layer):
            acc_f[i] = np.dot(self.weights[i], acc_g[i]) + self.biases[i]
            acc_g[i+1] = sigmoid(acc_f[i])

    def backprop_between_layers(self, start, end, acc_g, dLdg):
        # compute the gradients for layers [start, end)
        # dLdg is a gradient with respect to nonlinearity of layer[end-1]
        dLdWs = []
        dLdBs = []
        for w, g, prev_g in reversed(list(zip(self.weights[start:end], acc_g[start+1:end+1], acc_g[start:end]))):
            dLdf = dLdg * (1 - g) * g
            dLdWs.append(dLdf @ prev_g.T)
            dLdBs.append(dLdf)
            dLdg = w.T @ dLdf
        return reversed(dLdWs), reversed(dLdBs), dLdg
    
    def update_mini_batch(self, x_mini_batch, y_mini_batch, eta):
        # Update networks weights and biases by applying a single step
        # of gradient descent using backpropagation with checkpoints to compute the gradient.
        # For this exercise, we assume 1 element mini_batch
        # eta is the learning rate
        x_mini_batch = x_mini_batch.reshape(-1, 1)
        y_mini_batch = y_mini_batch.reshape(-1, 1)

        fs, gs, output = self.feedforward_with_checkpoints(x_mini_batch)
        dLdg = output - y_mini_batch
        for start, end in reversed(list(zip([-1] + self.checkpoints, self.checkpoints + [self.num_layers-1]))):
            # if checkpoints are (a, b) then we can backprop through layers [a+1, b] inclusive
            start += 1
            end += 1
            # those copies are inefficient, but we do them to keep indexing simple
            acc_f = fs.copy()
            acc_g = gs.copy()
            self.feedforward_between_layers(start, end, acc_f, acc_g) # this technically recalculates g for the last layer (it is a checkpoint), but nah, works
            nabla_w, nabla_b, dLdg = self.backprop_between_layers(start, end, acc_g, dLdg)
            self.weights[start:end] = [w - eta * dw for w, dw in zip(self.weights[start:end], nabla_w)]
            self.biases[start:end] = [b - eta * db for b, db in zip(self.biases[start:end], nabla_b)]
        
    
    def evaluate(self, x_test_data, y_test_data):
        # Count the number of correct answers for test_data
        test_results = [(np.argmax(self.feedforward(x_test_data[i].reshape(784,1))), np.argmax(y_test_data[i]))
                        for i in range(len(x_test_data))]
        # return accuracy
        return np.mean([int(x == y) for (x, y) in test_results])
    
    def cost_derivative(self, output_activations, y):
        return (output_activations-y) 
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        x_train, y_train = training_data
        if test_data:
            x_test, y_test = test_data
        for j in range(epochs):
            for i in range(x_train.shape[0] // mini_batch_size):
                x_mini_batch = x_train[i*mini_batch_size:(i*mini_batch_size + mini_batch_size)]
                y_mini_batch = y_train[i*mini_batch_size:(i*mini_batch_size + mini_batch_size)]
                self.update_mini_batch(x_mini_batch, y_mini_batch, eta)
            if test_data:
                print("Epoch: {0}, Accuracy: {1}".format(j, self.evaluate(x_test, y_test)))
            else:
                print("Epoch: {0}".format(j))

In [ ]:
network = NetworkWithCheckpoints([784,30, 30,10], checkpoints=[1])
network.SGD((x_train, y_train), epochs=10, mini_batch_size=1, eta=0.01, test_data=(x_test, y_test) ) # per-example descend is really slow, try vectorizing it!
# Just so you know, un-vectorized version takes about 25-35s per epoch 